## Clustering Neighborhoods in Toronto Project

### 1. Scrape data from wiki page

In [1]:
# import libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [7]:
wiki_cont = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").content
soup = BeautifulSoup(wiki_cont, 'html.parser')

In [8]:
table = soup.find('tbody')
rows = table.select('tr')
row = [r.get_text() for r in rows]

In [78]:
# get rid of empty element 
row2 = [list(filter(None,x.split('\n'))) for x in row if x.split('\n')]
df = pd.DataFrame(row2 )
df.columns = df.iloc[0]
df2 = df.iloc[1:,:]
df2.head(5)

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


### 2. Data cleaning and manipulation

In [83]:
# Ignore cells with a borough that is Not assigned
df3 = df2[df2['Borough'] != 'Not assigned']
# combine neightborhoods in same post code
df4 = df3.groupby(['Postcode', 'Borough']).agg(','.join).reset_index()

In [84]:
df4.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [89]:
# check how many records have 'not assigned' neighbourhood
df4[df4.Neighbourhood == 'Not assigned']

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


In [91]:
# replace 'Not assigned' neighbourhood to Queen's Park
df4['Neighbourhood'] = df4['Neighbourhood'].replace("Not assigned", "Queen's Park")

In [94]:
df4.shape

(103, 3)

## Include latitude and longtitude coordinate of each neighborhood (using csv files)

In [98]:
url = "http://cocl.us/Geospatial_data"
added_df = pd.read_csv(url)
added_df.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
added_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [100]:
# merge coordinate data into previous data
df5 = pd.merge(df4, added_df, on = 'Postcode', how = 'inner')
df5.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
